In [1]:
import requests
def query_ollama(prompt, model='llama3'):
    response = requests.post(
        'http://localhost:11434/api/generate',
        json={
            'model': model,
            'prompt': prompt,
            'stream': False
        }
    )
    return response.json()

In [2]:
import pandas as pd
df = pd.read_csv("data.csv")

In [11]:
def get_classification_prompt(message, previous_messages=""):
    prompt = f"""
You are a text classification model.

Given the following previous messages in the conversation and current message, first classify whether the current message is hate speech or not.

If it is hate speech, provide:
1. A primary label (one of the following):
   - IdentityDirectedAbuse
   - AffiliationDirectedAbuse
   - PersonDirectedAbuse
   - CounterSpeech
   - Slur

2. A secondary label, chosen based on the selected primary label:
   - If IdentityDirectedAbuse:
       • IdentityDirectedAbuse / animosity
       • IdentityDirectedAbuse / derogation
       • IdentityDirectedAbuse / glorification
       • IdentityDirectedAbuse / dehumanization
       • IdentityDirectedAbuse / threatening language
   - If AffiliationDirectedAbuse:
       • AffiliationDirectedAbuse / animosity
       • AffiliationDirectedAbuse / derogation
       • AffiliationDirectedAbuse / dehumanization
       • AffiliationDirectedAbuse / threatening language
   - If PersonDirectedAbuse:
       • PersonDirectedAbuse / directed to a person in the thread
       • PersonDirectedAbuse / about a person who is not in the thread
   - If CounterSpeech:
       • CounterSpeech / identity-directed
       • CounterSpeech / affiliation-directed
       • CounterSpeech / person-directed
   - If Slur:
       • (No secondary label required)

If it is NOT hate speech, reply only with:
Not Hate Speech

Previous Messages:
\"\"\"{previous_messages}\"\"\"

Current Message:
\"\"\"{message}\"\"\"
"""
    return prompt

In [12]:
valid_primary_labels = [
    "IdentityDirectedAbuse",
    "AffiliationDirectedAbuse",
    "PersonDirectedAbuse",
    "CounterSpeech",
    "Slur"
]

valid_secondary_labels = [
    "IdentityDirectedAbuse / animosity",
    "IdentityDirectedAbuse / derogation",
    "AffiliationDirectedAbuse / animosity",
    "AffiliationDirectedAbuse / derogation",
    "PersonDirectedAbuse / directed to a person in the thread",
    "PersonDirectedAbuse / about a person who is not in the thread",
    "CounterSpeech / identity-directed",
    "CounterSpeech / affiliation-directed",
    "CounterSpeech / person-directed",
    "IdentityDirectedAbuse / glorification",
    "AffiliationDirectedAbuse / dehumanization",
    "IdentityDirectedAbuse / dehumanization",
    "IdentityDirectedAbuse / threatening language",
    "AffiliationDirectedAbuse / threatening language"
]

In [15]:
import numpy as np
import ast
def classify_and_update_labels(index):
    message = df.at[index, 'utterance']
    context = ast.literal_eval(df.at[index, 'context'])
    if len(context) == 0:
        previous_message = ''
    else:
        previous_message = "\n".join(context)
    
    prompt = get_classification_prompt(message, previous_message)
    response = query_ollama(prompt)['response'].strip()

    primary_label = None
    secondary_label = None

    if "Not Hate Speech" in response:
        primary_label = "Not Hate Speech"
        secondary_label = np.nan
    else:
        for label in valid_primary_labels:
            if label in response:
                primary_label = label
                break
        if not primary_label:
            print(f"No valid primary label found at index {index}")
            failed_indices.append(index)
        
        if primary_label!='Slur':
            for label in valid_secondary_labels:
                if label in response:
                    secondary_label = label
                    break
            if not secondary_label:
                print(f"No valid secondary label found at index {index}")
                if failed_indices[-1]!=index:
                    failed_indices.append(index)

    # Save to prediction columns
    df.at[index, 'pred_primary'] = primary_label
    df.at[index, 'pred_secondary'] = secondary_label

In [16]:
non_neutral_indices = df[df['label_primary'] != 'Neutral'].index.tolist()

In [19]:
failed_indices=[-1]

In [20]:
from tqdm import tqdm

for index in tqdm(non_neutral_indices[:1000], desc="Processing Non-Neutral Rows"):
    classify_and_update_labels(index)

Processing Non-Neutral Rows:   1%|▎                                                   | 6/1000 [00:16<44:28,  2.68s/it]

No valid primary label found at index 278
No valid secondary label found at index 278


Processing Non-Neutral Rows:   2%|▊                                                  | 15/1000 [00:38<42:17,  2.58s/it]

No valid primary label found at index 369
No valid secondary label found at index 369


Processing Non-Neutral Rows:   2%|▉                                                  | 19/1000 [00:48<41:40,  2.55s/it]

...

Processing Non-Neutral Rows:  99%|█████████████████████████████████████████████████▋| 993/1000 [46:08<00:18,  2.58s/it]

No valid primary label found at index 20175
No valid secondary label found at index 20175


Processing Non-Neutral Rows: 100%|█████████████████████████████████████████████████▉| 998/1000 [46:22<00:06,  3.11s/it]

No valid secondary label found at index 20181


Processing Non-Neutral Rows: 100%|█████████████████████████████████████████████████| 1000/1000 [46:27<00:00,  2.79s/it]


In [21]:
df.to_csv('outputs.csv', index=False)

In [26]:
redo_indices=failed_indices[1:]

In [32]:
import pickle
with open('redo.pkl', 'wb') as f:
    pickle.dump(redo_indices, f)

In [34]:
failed_indices=redo_indices

In [36]:
for index in tqdm(non_neutral_indices[1000:2000], desc="Processing Non-Neutral Rows"):
    classify_and_update_labels(index)

Processing Non-Neutral Rows:   0%|                                                    | 1/1000 [00:03<59:15,  3.56s/it]

No valid secondary label found at index 20184


Processing Non-Neutral Rows:   0%|                                                    | 2/1000 [00:06<49:30,  2.98s/it]

No valid primary label found at index 20185
No valid secondary label found at index 20185


Processing Non-Neutral Rows:   1%|▋                                                  | 14/1000 [00:36<44:06,  2.68s/it]

...

No valid primary label found at index 21268
No valid secondary label found at index 21268


Processing Non-Neutral Rows: 100%|█████████████████████████████████████████████████▉| 999/1000 [46:34<00:02,  2.87s/it]

No valid primary label found at index 21269
No valid secondary label found at index 21269


Processing Non-Neutral Rows: 100%|█████████████████████████████████████████████████| 1000/1000 [46:37<00:00,  2.80s/it]

No valid primary label found at index 21270
No valid secondary label found at index 21270


In [39]:
df.to_csv('outputs.csv', index=False)

In [41]:
redo_indices=failed_indices

In [42]:
with open('redo.pkl', 'wb') as f:
    pickle.dump(redo_indices, f)

In [43]:
for index in tqdm(non_neutral_indices[2000:3000], desc="Processing Non-Neutral Rows"):
    classify_and_update_labels(index)

Processing Non-Neutral Rows:   1%|▎                                                   | 7/1000 [00:19<44:48,  2.71s/it]

No valid primary label found at index 21277
No valid secondary label found at index 21277


Processing Non-Neutral Rows:   1%|▌                                                  | 12/1000 [00:32<42:52,  2.60s/it]

No valid primary label found at index 21282
No valid secondary label found at index 21282


Processing Non-Neutral Rows:   2%|▊                                                  | 16/1000 [00:42<43:24,  2.65s/it]

...

Processing Non-Neutral Rows:  99%|█████████████████████████████████████████████████▌| 991/1000 [45:30<00:23,  2.63s/it]

No valid primary label found at index 22410
No valid secondary label found at index 22410


Processing Non-Neutral Rows: 100%|█████████████████████████████████████████████████▊| 997/1000 [45:46<00:07,  2.60s/it]

No valid primary label found at index 22416
No valid secondary label found at index 22416


Processing Non-Neutral Rows: 100%|█████████████████████████████████████████████████| 1000/1000 [45:53<00:00,  2.75s/it]


In [44]:
df.to_csv('outputs.csv', index=False)

In [46]:
redo_indices=failed_indices

In [47]:
with open('redo.pkl', 'wb') as f:
    pickle.dump(redo_indices, f)

In [50]:
for index in tqdm(non_neutral_indices[3000:4000], desc="Processing Non-Neutral Rows"):
    classify_and_update_labels(index)

Processing Non-Neutral Rows:   0%|                                                    | 1/1000 [00:03<54:21,  3.27s/it]

No valid primary label found at index 22420
No valid secondary label found at index 22420


Processing Non-Neutral Rows:   0%|▏                                                   | 4/1000 [00:11<47:31,  2.86s/it]

No valid primary label found at index 22423
No valid secondary label found at index 22423


Processing Non-Neutral Rows:   0%|▎                                                   | 5/1000 [00:14<45:45,  2.76s/it]

...

Processing Non-Neutral Rows:  98%|█████████████████████████████████████████████████ | 981/1000 [46:33<00:52,  2.76s/it]

No valid primary label found at index 23507
No valid secondary label found at index 23507


Processing Non-Neutral Rows: 100%|█████████████████████████████████████████████████▉| 998/1000 [47:27<00:06,  3.12s/it]

No valid secondary label found at index 23526


Processing Non-Neutral Rows: 100%|█████████████████████████████████████████████████| 1000/1000 [47:32<00:00,  2.85s/it]


In [51]:
df.to_csv('outputs.csv', index=False)

In [52]:
redo_indices=failed_indices

In [53]:
with open('redo.pkl', 'wb') as f:
    pickle.dump(redo_indices, f)

In [54]:
for index in tqdm(non_neutral_indices[4000:5000], desc="Processing Non-Neutral Rows"):
    classify_and_update_labels(index)

Processing Non-Neutral Rows:   1%|▎                                                   | 7/1000 [00:22<51:15,  3.10s/it]

No valid primary label found at index 23535
No valid secondary label found at index 23535


Processing Non-Neutral Rows:   3%|█▋                                                 | 32/1000 [01:29<42:36,  2.64s/it]

No valid primary label found at index 23562
No valid secondary label found at index 23562


Processing Non-Neutral Rows:   4%|█▉                                                 | 37/1000 [01:45<49:51,  3.11s/it]

...

Processing Non-Neutral Rows:  99%|█████████████████████████████████████████████████▋| 993/1000 [46:47<00:19,  2.80s/it]

No valid secondary label found at index 24635


Processing Non-Neutral Rows: 100%|█████████████████████████████████████████████████▉| 999/1000 [47:04<00:03,  3.07s/it]

No valid primary label found at index 24641
No valid secondary label found at index 24641


Processing Non-Neutral Rows: 100%|█████████████████████████████████████████████████| 1000/1000 [47:07<00:00,  2.83s/it]


In [55]:
df.to_csv('outputs.csv', index=False)

In [56]:
redo_indices=failed_indices

In [58]:
with open('redo.pkl', 'wb') as f:
    pickle.dump(redo_indices, f)

In [59]:
for index in tqdm(non_neutral_indices[5000:], desc="Processing Non-Neutral Rows"):
    classify_and_update_labels(index)

Processing Non-Neutral Rows:   0%|▏                                                    | 2/426 [00:05<19:35,  2.77s/it]

No valid primary label found at index 24644
No valid secondary label found at index 24644


Processing Non-Neutral Rows:   2%|█▏                                                  | 10/426 [00:27<18:45,  2.71s/it]

No valid primary label found at index 24653
No valid secondary label found at index 24653


Processing Non-Neutral Rows:   4%|█▉                                                  | 16/426 [00:42<17:34,  2.57s/it]

...

Processing Non-Neutral Rows:  98%|█████████████████████████████████████████████████▊ | 416/426 [18:55<00:25,  2.56s/it]

No valid primary label found at index 26315
No valid secondary label found at index 26315


Processing Non-Neutral Rows:  99%|██████████████████████████████████████████████████▋| 423/426 [19:14<00:08,  2.85s/it]

No valid primary label found at index 26322
No valid secondary label found at index 26322


Processing Non-Neutral Rows: 100%|███████████████████████████████████████████████████| 426/426 [19:22<00:00,  2.73s/it]


In [60]:
df.to_csv('outputs.csv', index=False)

In [61]:
redo_indices=failed_indices

In [62]:
with open('redo.pkl', 'wb') as f:
    pickle.dump(redo_indices, f)

In [63]:
preds = df.iloc[non_neutral_indices]

In [95]:
preds['pred_primary'] = preds['pred_primary'].fillna('Hate Speech')

C:\Users\manda\AppData\Local\Temp\ipykernel_16364\3363166466.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preds['pred_primary'] = preds['pred_primary'].fillna('Hate Speech')


In [96]:
preds

,utterance,context,label_primary,label_secondary,context_needed,pred_primary,pred_secondary
90,No Republican can ever be trusted. Mueller is ...,['Throwaway02122019: Remember that Robert Muel...,AffiliationDirectedAbuse,AffiliationDirectedAbuse / derogation,0.0,PersonDirectedAbuse,PersonDirectedAbuse / directed to a person in ...
216,"The original ""Hey, would you like cake or pie?...","[""pyrbear: Yes is the laziest, tired, and most...",Slur,NaN,0.0,Not Hate Speech,NaN
262,What a fucking idiot. [linebreak] [linebreak]...,"['butterflyonsteroids: Weird flex but okkk', '...",PersonDirectedAbuse,PersonDirectedAbuse / about a person who is no...,0.0,PersonDirectedAbuse,PersonDirectedAbuse / directed to a person in ...
269,Easy there Hitler.,"['butterflyonsteroids: Weird flex but okkk', '...",PersonDirectedAbuse,PersonDirectedAbuse / directed to a person in ...,0.0,PersonDirectedAbuse,PersonDirectedAbuse / directed to a person in ...
274,Oof. That's dumb. Makes actaul people with dys...,"['footfaceball: UwU', 'footfaceball:', 'CritFa...",IdentityDirectedAbuse,IdentityDirectedAbuse / animosity,1.0,PersonDirectedAbuse,PersonDirectedAbuse / directed to a person in ...
...,...,...,...,...,...,...,...
26321,Seems to me that POC are more privileged than ...,[],IdentityDirectedAbuse,IdentityDirectedAbuse / animosity,0.0,IdentityDirectedAbuse,IdentityDirectedAbuse / derogation
26322,#Victims There is now the possibility that the...,[],AffiliationDirectedAbuse,AffiliationDirectedAbuse / derogation,0.0,Hate Speech,None
26323,Drinking too much hot water from your Rainies?,[],IdentityDirectedAbuse,IdentityDirectedAbuse / animosity,0.0,Not Hate Speech,NaN
26324,Night ninjas,[],IdentityDirectedAbuse,IdentityDirectedAbuse / derogation,0.0,Not Hate Speech,NaN


In [97]:
count = (preds['pred_primary'] == 'Not Hate Speech').sum()
accuracy=1-count/len(preds)
print("Accuracy for binary classification: ",accuracy)

Accuracy for binary classification:  0.659049023221526


In [91]:
from sklearn.metrics import classification_report

In [98]:
y_true = preds['label_primary']      
y_pred = preds['pred_primary']

In [99]:
report = classification_report(y_true, y_pred)
print(report)

C:\Users\manda\anaconda3\envs\new_env_name\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\manda\anaconda3\envs\new_env_name\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                          precision    recall  f1-score   support

AffiliationDirectedAbuse       0.53      0.01      0.01      1327
           CounterSpeech       0.00      0.00      0.00       220
             Hate Speech       0.00      0.00      0.00         0
   IdentityDirectedAbuse       0.50      0.08      0.14      2635
         Not Hate Speech       0.00      0.00      0.00         0
     PersonDirectedAbuse       0.25      0.43      0.31      1100
                    Slur       0.07      0.06      0.06       144

                accuracy                           0.13      5426
               macro avg       0.19      0.08      0.08      5426
            weighted avg       0.42      0.13      0.14      5426



C:\Users\manda\anaconda3\envs\new_env_name\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
